In [30]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report

In [23]:
input_path = 'treatments/'

dataset = pd.read_csv(input_path + 'treatment_class.csv')
display(dataset.shape, dataset.head())

(138, 6)

,treatments,Toxic,Lower Effect,Deductive-Toxic,Deductive-Lower Effect,Emergency
0,"Carboplatin, Paclitaxel, Atorvastatin, Levoflo...",0.400000,0.100000,0.500000,0.100000,12.0
1,"Carboplatin, Etoposide, Atorvastatin, Enalapri...",0.266667,0.066667,0.266667,0.066667,8.0
2,"Cisplatin, Pemetrexed, Vinorelbine, Lorazepam,...",0.533333,0.200000,0.733333,0.200000,7.0
3,"Carboplatin, Etoposide, Ibuprofen, Omeprazole,...",0.300000,0.100000,0.300000,0.100000,6.0
4,"Carboplatin, Pemetrexed, Dexamethasone, Loraze...",0.400000,0.400000,0.600000,0.500000,5.0


In [24]:
t = 'Carboplatin, Etoposide, Atorvastatin, Enalapril, Metformin, Omeprazole'
dataset = dataset.loc[~((dataset.treatments==t) & (dataset.Emergency==1))]

In [25]:
dataset.loc[dataset.Emergency > 2, 'class'] = 1
dataset.loc[dataset.Emergency <= 2, 'class'] = 0

In [26]:
target_class = dataset[['treatments', 'class']]
target_class.drop_duplicates(keep='first', inplace=True)
target_class.reset_index(drop=True, inplace=True)

data = dataset.drop(['Toxic', 'Lower Effect', 'Emergency', 'class'], axis=1)
data.drop_duplicates(keep='first', inplace=True)
data.reset_index(drop=True, inplace=True)

display(target_class.shape, target_class.head(2), data.shape, data.head(2))

/home/rivas/.local/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(133, 2)

,treatments,class
0,"Carboplatin, Paclitaxel, Atorvastatin, Levoflo...",1.0
1,"Carboplatin, Etoposide, Atorvastatin, Enalapri...",1.0


(133, 3)

,treatments,Deductive-Toxic,Deductive-Lower Effect
0,"Carboplatin, Paclitaxel, Atorvastatin, Levoflo...",0.500000,0.100000
1,"Carboplatin, Etoposide, Atorvastatin, Enalapri...",0.266667,0.066667


In [27]:
target_class['class'].value_counts()

0.0    111
1.0     22
Name: class, dtype: int64

In [28]:
new_data = data.drop(columns=['treatments'])

In [32]:
X_train, X_test, y_train, y_test = train_test_split(new_data.values, target_class['class'].values, random_state=42)

In [33]:
feature_names = new_data.columns
parameters = {"max_depth": range(1,7)}
clf = tree.DecisionTreeClassifier()
grid = GridSearchCV(estimator=clf,param_grid=parameters)
grid_res = grid.fit(X_train, y_train)
best_clf = grid_res.best_estimator_


#predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_clf.score(X_test, y_test)
y_pred = best_clf.predict(X_test)

In [41]:
target_names = ['Effectiveness','Low Effect']
print (classification_report(y_test, y_pred, target_names=target_names))

               precision    recall  f1-score   support

Effectiveness       0.76      1.00      0.87        26
   Low Effect       0.00      0.00      0.00         8

     accuracy                           0.76        34
    macro avg       0.38      0.50      0.43        34
 weighted avg       0.58      0.76      0.66        34



/home/rivas/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rivas/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/rivas/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
import dtreeviz

viz = dtreeviz(best_clf, new_data, target_class['class'], target_name='class',
               feature_names=feature_names, class_names=target_names, fancy=False,
               show_root_edge_labels=True)

TypeError: 'module' object is not callable